# Smartnoise-sql

In [1]:
from snsql import Privacy, from_connection, Stat, Mechanism
import pandas as pd

In [2]:
metadata = {'': {'Schema': {'Table': {'max_ids': 1,
    'row_privacy': True,
    'censor_dims': False,
    'species': {'type': 'string',
     'cardinality': 3,
     'categories': ['Adelie', 'Chinstrap', 'Gentoo']},
    'island': {'type': 'string',
     'cardinality': 3,
     'categories': ['Torgersen', 'Biscoe', 'Dream']},
    'bill_length_mm': {'type': 'float', 'lower': 30.0, 'upper': 65.0},
    'bill_depth_mm': {'type': 'float', 'lower': 13.0, 'upper': 23.0},
    'flipper_length_mm': {'type': 'float', 'lower': 150.0, 'upper': 250.0},
    'body_mass_g': {'type': 'float', 'lower': 2000.0, 'upper': 7000.0},
    'sex': {'type': 'string',
     'cardinality': 2,
     'categories': ['MALE', 'FEMALE']}}}},
 'engine': 'csv'}

In [3]:
df = pd.read_csv("https://raw.githubusercontent.com/mwaskom/seaborn-data/master/penguins.csv")
df.head()

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,MALE
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,FEMALE
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,FEMALE
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,FEMALE


In [4]:
grouped = df.groupby('species').size().reset_index(name='count')
grouped

,species,count
0,Adelie,152
1,Chinstrap,68
2,Gentoo,124


In [5]:
privacy = Privacy(epsilon=1.0, delta=0.0001)
privacy.mechanisms.map[Stat['count']] = Mechanism["discrete_gaussian"]

In [6]:
reader = from_connection(df, privacy=privacy, metadata=metadata)

/usr/local/lib/python3.8/site-packages/snsql/sql/private_reader.py:136: UserWarning: The following columns are of type float: bill_length_mm, bill_depth_mm, flipper_length_mm, body_mass_g. 
summary statistics over floats will use Mechanism.laplace, which is not floating-point safe, 
This could lead to privacy leaks.
  warnings.warn(


In [7]:
QUERY = "SELECT \
        species AS species, \
        COUNT(*) AS nb_penguin,  \
        AVG(bill_length_mm) AS avg_bill_length_mm, \
        STD(bill_length_mm) AS std_bill_length_mm \
        FROM Schema.Table  GROUP BY species"

In [8]:
QUERY = "SELECT AVG(bill_length_mm) AS avg_bill_length_mm FROM Schema.Table"

## Pre agg, port process and mechanisms

In [9]:
pre_agg = [
    ['species', 'count_star'],
    ['Adelie', 152],
    ['Chinstrap', 68],
    ['Gentoo', 124]
]

In [24]:
query = "SELECT species AS species, COUNT(*) AS nb_penguin FROM Schema.Table GROUP BY species"

In [25]:
result = reader.get_privacy_cost(query)
result

(1.0, 0.00014999500000001387)

In [26]:
result = reader.execute(query, postprocess=False, pre_aggregated = pre_agg)
list(result)

[['Adelie', 152], ['Chinstrap', 69], ['Gentoo', 126]]

In [27]:
query = "SELECT species AS species, COUNT(*) AS nb_penguin FROM Schema.Table GROUP BY species"
result = reader.execute(query, postprocess=True, pre_aggregated=pre_agg)
list(result)

[['species', 'nb_penguin'],
 ['Adelie', 149],
 ['Chinstrap', 63],
 ['Gentoo', 126]]

In [12]:
mechanisms = {
    "count": "discrete_gaussian"
}
STATS = ["count", "sum_int", "sum_large_int", "sum_float", "threshold"]

In [13]:
for stat in STATS:
    if stat in mechanisms.keys():
        privacy.mechanisms.map[Stat[stat]] = Mechanism[mechanisms[stat]]

In [14]:
privacy = Privacy(epsilon=1.0)
print(f"We default to using {privacy.mechanisms.map[Stat.count]} for counts.")
print("Switching to use gaussian")
privacy.mechanisms.map[Stat['count']] = Mechanism["discrete_gaussian"]

We default to using Mechanism.discrete_laplace for counts.
Switching to use gaussian


### See expression and budget

In [15]:
def get_expressions(query):
    subquery, _ = reader._rewrite(query)
    expressions = [
        f'{str(col.expression)} AS {col.name}'
        for col in subquery.select.namedExpressions
    ]
    return expressions

In [16]:
query = "SELECT species AS species, COUNT(*) AS nb_penguin FROM Schema.Table GROUP BY species"
print(reader.get_privacy_cost(query))
get_expressions(query)

(1.0, 0.00014999500000001387)


['species AS species', 'COUNT(*) AS count_star']

In [17]:
query = "SELECT species AS species, SUM(bill_length_mm) * SUM(bill_length_mm) AS sum_squared FROM Schema.Table GROUP BY species"
print(reader.get_privacy_cost(query))
get_expressions(query)

(1.0, 4.999999999999449e-05)


['species AS species', 'SUM(bill_length_mm) AS sum_bill_length_mm']

In [18]:
query = "SELECT species AS species, AVG(bill_length_mm) AS avg_bill_length_mm FROM Schema.Table GROUP BY species"
print(reader.get_privacy_cost(query))
get_expressions(query)

(2.0, 0.00014999500000001387)


['species AS species',
 'SUM(bill_length_mm) AS sum_bill_length_mm',
 'COUNT(bill_length_mm) AS count_bill_length_mm']

In [19]:
query = "SELECT STD(bill_length_mm) AS std_bill_length_mm FROM Schema.Table GROUP BY species"
print(reader.get_privacy_cost(query))
get_expressions(query)

(3.0, 0.00014999500000001387)


['species AS species',
 'SUM(bill_length_mm * bill_length_mm) AS sum_alias_0xd6d2',
 'COUNT(bill_length_mm) AS count_bill_length_mm',
 'SUM(bill_length_mm) AS sum_bill_length_mm']

In [20]:
query = "SELECT species AS species, STD(bill_length_mm) AS std_bill_length_mm FROM Schema.Table GROUP BY species"
print(reader.get_privacy_cost(query))
get_expressions(query)

(3.0, 0.00014999500000001387)


['species AS species',
 'SUM(bill_length_mm * bill_length_mm) AS sum_alias_0xd6d2',
 'COUNT(bill_length_mm) AS count_bill_length_mm',
 'SUM(bill_length_mm) AS sum_bill_length_mm']

In [21]:
query = "SELECT \
        species AS species, \
        COUNT(*) AS nb_penguin,  \
        AVG(bill_length_mm) AS avg_bill_length_mm, \
        STD(bill_length_mm) AS std_bill_length_mm \
        FROM Schema.Table  GROUP BY species"
print(reader.get_privacy_cost(query))
get_expressions(query)

(4.0, 0.0002499800004999164)


['species AS species',
 'COUNT(*) AS count_star',
 'SUM(bill_length_mm) AS sum_bill_length_mm',
 'COUNT(bill_length_mm) AS count_bill_length_mm',
 'SUM(bill_length_mm * bill_length_mm) AS sum_alias_0xd6d2']

In [22]:
query = "SELECT \
        species AS species, \
        COUNT(bill_length_mm) AS nb_penguin,  \
        AVG(bill_length_mm) AS avg_bill_length_mm, \
        STD(bill_length_mm) AS std_bill_length_mm \
        FROM Schema.Table  GROUP BY species"
print(reader.get_privacy_cost(query))
get_expressions(query)

(3.0, 0.00014999500000001387)


['species AS species',
 'COUNT(bill_length_mm) AS count_bill_length_mm',
 'SUM(bill_length_mm) AS sum_bill_length_mm',
 'SUM(bill_length_mm * bill_length_mm) AS sum_alias_0xd6d2']

### Multiple queries

In [29]:
QUERY

'SELECT AVG(bill_length_mm) AS avg_bill_length_mm FROM Schema.Table'

In [30]:
query = "SELECT species AS species, COUNT(*) AS nb_penguin FROM Schema.Table GROUP BY species"
result = reader.execute([QUERY, query])
result

TypeError: ord() expected a character, but string of length 66 found

## Via Server

In [108]:
from fso_sdd_demo.client import Client
import numpy as np

In [109]:
APP_URL = "http://sdd_server_dev:80"
USER_NAME = "Dr. Antartica"
DATASET_NAME = "PENGUIN"
client = Client(url=APP_URL, user_name = USER_NAME, dataset_name = DATASET_NAME)

In [110]:
metadata = client.get_dataset_metadata()
metadata

{'': {'Schema': {'Table': {'max_ids': 1,
    'row_privacy': True,
    'censor_dims': False,
    'species': {'type': 'string',
     'cardinality': 3,
     'categories': ['Adelie', 'Chinstrap', 'Gentoo']},
    'island': {'type': 'string',
     'cardinality': 3,
     'categories': ['Torgersen', 'Biscoe', 'Dream']},
    'bill_length_mm': {'type': 'float', 'lower': 30.0, 'upper': 65.0},
    'bill_depth_mm': {'type': 'float', 'lower': 13.0, 'upper': 23.0},
    'flipper_length_mm': {'type': 'float', 'lower': 150.0, 'upper': 250.0},
    'body_mass_g': {'type': 'float', 'lower': 2000.0, 'upper': 7000.0},
    'sex': {'type': 'string',
     'cardinality': 2,
     'categories': ['MALE', 'FEMALE']}}}},
 'engine': 'csv'}

In [154]:
# QUERY = "SELECT AVG(bill_length_mm) AS avg_bill_length_mm FROM Schema.Table" #  {"sum_int": "laplace"}
# QUERY = "SELECT COUNT(*) AS NB_ROW FROM Schema.Table" # {"count": "discrete_gaussian"}
QUERY = "SELECT \
        species AS species, \
        COUNT(*) AS nb_penguin,  \
        AVG(bill_length_mm) AS avg_bill_length_mm, \
        STD(bill_length_mm) AS std_bill_length_mm \
        FROM Schema.Table  GROUP BY species"

In [164]:
NB_ROWS = 200
SEED = 0

res = client.smartnoise_query(
    query = QUERY,  
    epsilon = 100.0,
    delta = 2.0,
    mechanisms = {"sum_int": "laplace"},
    postprocess = True,
    dummy = True, 
    nb_rows = NB_ROWS,
    seed = SEED
)

In [165]:
res_server_dummy = res['query_response']
res_server_dummy

,species,nb_penguin,avg_bill_length_mm,std_bill_length_mm
0,Adelie,58,47.120315,9.999328
1,Chinstrap,75,47.895683,10.137082
2,Gentoo,67,47.391628,9.918179


In [167]:
client.estimate_smartnoise_cost(
    query = QUERY, 
    epsilon = 1.0, 
    delta = 1e-4,
    mechanisms = {"sum_int": "laplace"}
)

{'epsilon_cost': 4.0, 'delta_cost': 0.00014999500000001387}

In [178]:
res = client.smartnoise_query(
    query = QUERY,  
    epsilon = 0.2,
    delta = 1e-5,
    mechanisms = {"sum_int": "laplace"},
    #postprocess = True,
    dummy = True, 
    nb_rows = NB_ROWS,
    seed = SEED
)

In [179]:
res['query_response']

,species,nb_penguin,avg_bill_length_mm,std_bill_length_mm
0,Adelie,44,37.383913,37.213453
1,Chinstrap,77,46.621092,21.515840
2,Gentoo,77,36.947055,35.157860


In [196]:
EPSILON = 0.2
DELTA = 1e-5

avg_bill_length_response = client.smartnoise_query(
    query = QUERY,  
    epsilon = EPSILON, 
    delta = DELTA,
    mechanisms = {"sum_int": "laplace"},
    #postprocess = True
)

In [197]:
avg_bill_length_response['query_response']

,species,nb_penguin,avg_bill_length_mm,std_bill_length_mm
0,Adelie,149,35.658620,25.310481
1,Chinstrap,68,36.093032,33.664268
2,Gentoo,123,46.702786,9.648210
